The source base for *A Gospel of Health and Salvation* is the collection of scanned periodicals produced by the [Office of Archives, Statistics, and Research](http://documents.adventistarchives.org/) of the Seventh-day Adventist Church (SDA). That this collection of documents is openly available on the web has been fundamental to the success of this project. One of the greatest challenges for historical scholarship that seeks to leverage large digital collections is access to the relevant materials. While projects such as [Chronicling America](http://chroniclingamerica.loc.gov/) and resources such as the [Digital Public Library of America](http://dp.la) are indispensable, many specialized resources are available only through proprietary databases and library subscriptions that impose limits on the ways scholars can interact with their resources.[^1] 

The publishing of the digital periodicals on the open web by the SDA made it unnecessary for me to navigate through the firewalls (and legal land-mines) of using text from major library databases, a major boon for the digital project.[^3] And, although the site does not provide an API for accessing the documents, the structure of the pages is regular, making the site a good candidate for web scraping. However, relying on an organization to provide its own historical documents raises its own challenges. Due to the interests of the hosting organization, in this case the Seventh-day Adventist Church, the collection is shaped by and shapes a particular narrative of the denomination's history and development. For example, issues of *Good Health*, which was published by John Harvey Kellogg, are (almost entirely) dropped from the SDA's collection after 1907, which corresponds to the point when Kellogg was disfellowshipped from the denomination, even though Kellogg continued its publication into the 1940s.[^2] Such interests do not invalidate the usefulness of the collection, as all archives have limitations and goals, but those interests need to be acknowledged and taken into account in the analysis.

To determine the list of titles that applied to my time and regions of study, I browsed through all of the titles in the [periodicals section of the site](http://documents.adventistarchives.org/Periodicals/Forms/AllFolders.aspx) and compiled a list of titles that fit my geographic and temporal constraints. These are: 

* [Training School Advocate (ADV)](http://documents.adventistarchives.org/Periodicals/ADV)
* [American Sentinel (AmSn)](http://documents.adventistarchives.org/Periodicals/AmSn)
* [Advent Review and Sabbath Herald (ARAI)](http://documents.adventistarchives.org/Periodicals/ARAI)
* [Christian Education (CE)](http://documents.adventistarchives.org/Periodicals/CE)
* [Welcome Visitor (Columbia Union Visitor) (CUV)](http://documents.adventistarchives.org/Periodicals/CUV)
* [Christian Educator (EDU)](http://documents.adventistarchives.org/Periodicals/EDU)
* [General Conference Bulletin (GCB)](http://documents.adventistarchives.org/Periodicals/GCSessionBulletins)
* [Gospel Herald (GH)](http://documents.adventistarchives.org/Periodicals/GH)
* [Gospel of Health (GOH)](http://documents.adventistarchives.org/Periodicals/GOH)
* [Gospel Sickle (GS)](http://documents.adventistarchives.org/Periodicals/GS)
* [Home Missionary (HM)](http://documents.adventistarchives.org/Periodicals/HM)
* [Health Reformer (HR)](http://documents.adventistarchives.org/Periodicals/HR)
* [Indiana Reporter (IR)](http://documents.adventistarchives.org/Periodicals/IR)
* [Life Boat (LB)](http://documents.adventistarchives.org/Periodicals/LB)
* [Life and Health (LH)](http://documents.adventistarchives.org/Periodicals/LH)
* [Liberty (LibM)](http://documents.adventistarchives.org/Periodicals/LibM)
* [Lake Union Herald (LUH)](http://documents.adventistarchives.org/Periodicals/LUH)
* [North Michigan News Sheet (NMN)](http://documents.adventistarchives.org/Periodicals/NMN)
* [Pacific Health Journal and Temperance Advocate (PHJ)](http://documents.adventistarchives.org/Periodicals/PHJ)
* [Present Truth (Advent Review) (PT-AR)](http://documents.adventistarchives.org/Periodicals/PT-AR) (renamed locally to PTAR)
* [Pacific Union Recorder (PUR)](http://documents.adventistarchives.org/Periodicals/PUR)
* [Review and Herald (RH)](http://documents.adventistarchives.org/Periodicals/RH)
* [Sabbath School Quarterly (SSQ)](http://documents.adventistarchives.org/SSQ)
* [Sligonian (Sligo)](http://documents.adventistarchives.org/Periodicals/Sligo)
* [Sentinel of Liberty (SOL)](http://documents.adventistarchives.org/Periodicals/SOL)
* [Signs of the Times (ST)](http://documents.adventistarchives.org/Periodicals/ST)
* [Report of Progress, Southern Union Conference (SUW)](http://documents.adventistarchives.org/Periodicals/SUW)
* [The Church Officer's Gazette (TCOG)](http://documents.adventistarchives.org/Periodicals/TCOG)
* [The Missionary Magazine (TMM)](http://documents.adventistarchives.org/Periodicals/TMM)
* [West Michigan Herald (WMH)](http://documents.adventistarchives.org/Periodicals/WMH)
* [Youth's Instructor (YI)](http://documents.adventistarchives.org/Periodicals/YI)

As this was my first technical task for the dissertation, my initial methods for identifying the URLs for the documents I wanted to download was rather manual. I saved an .html file for each index page that contained documents I wanted to download. I then passed those .html files to a script (similar to that recorded here) that used `BeautifulSoup` to extract the PDF ids, reconstruct the URLs, and write the URLs to a new text file, `scrapeList.txt`. After manually deleting the URLs to any documents that were out of range, I then passed the `scrapeList.txt` file to `wget` using the following syntax:[^4] 

```bash
wget -i scrapeList.txt -w 2 --limit-rate=200k
```

I ran this process for each of the periodical titles included in this study. It took approximately a week to download all 13,000 files to my local machine. The resulting corpus takes up 27.19 GB of space.

This notebook reflects a more automated version of that process, created in 2017 to download missing documents. The example recorded here is for downloading the [Sabbath School Quarterly](http://documents.adventistarchives.org/SSQ/) collection, which I missed during my initial collection phase. 

In these scripts I use the [`requests`](http://docs.python-requests.org/en/master/) library to retrieve the HTML from the document directory pages and [`BeautifulSoup4`](https://www.crummy.com/software/BeautifulSoup/) to locate the filenames. Finally, I use `requests` to download the files.

In [3]:
from bs4 import BeautifulSoup
from os.path import join
import re
import requests

In [4]:
def check_year(pdfID):
    """Use regex to check the year from the PDF filename.

    Args:
        pdfID (str): The filename of the PDF object, formatted as 
            PREFIXYYYYMMDD-V00-00
    """
    split_title = pdfID.split('-')
    title_date = split_title[0]
    date = re.findall(r'[0-9]+', title_date)
    year = date[0][:4]
    if int(year) < 1921:
        return True
    else:
        return False


def filename_from_html(content):
    """Use BeautifulSoup to extract the PDF ids from the HTML page. 

    This script is customized to the structure of the archive pages at
    http://documents.adventistarchives.org/Periodicals/Forms/AllFolders.aspx.

    Args:
        content (str): Content is retrieved from a URL using the `get_html_page` 
            function.
    """
    soup = BeautifulSoup(content, "lxml")
    buttons = soup.find_all('td', class_="ms-vb-title")

    pdfIDArray = []

    for each in buttons:
        links = each.find('a')
        pdfID = links.get_text()
        pdfIDArray.append(pdfID)

    return pdfIDArray


def get_html_page(url):
    """Use the requests library to get HTML content from URL
    
    Args:
        url (str): URL of webpage with content to download.
    """
    r = requests.get(url)

    return r.text

The first step is to set the directory where I want to save the downloaded documents, as well as the root URL for the location of the PDF documents. 

This example is set up for the Sabbath School Quarterly.

In [6]:
"""If running locally, you will need to create the `corpus` folder or 
update the path to the location of your choice.
"""
download_directory = "/Users/jeriwieringa/Desktop/test"
baseurl = "http://documents.adventistarchives.org/SSQ/"

My next step is to generate a list of the IDs for the documents I want to download. 

Here I download the HTML from the index page URLs and extract the document IDs. To avoid downloading any files outside of my study, I check the year in the ID before adding the document ID to my list of documents to download.

In [7]:
index_page_urls = ["http://documents.adventistarchives.org/SSQ/Forms/AllItems.aspx?View={44c9b385-7638-47af-ba03-cddf16ec3a94}&SortField=DateTag&SortDir=Asc",
              "http://documents.adventistarchives.org/SSQ/Forms/AllItems.aspx?Paged=TRUE&p_SortBehavior=0&p_DateTag=1912-10-01&p_FileLeafRef=SS19121001-04%2epdf&p_ID=457&PageFirstRow=101&SortField=DateTag&SortDir=Asc&&View={44C9B385-7638-47AF-BA03-CDDF16EC3A94}"
             ]

In [8]:
docs_to_download = []

for url in index_page_urls: 
    content = get_html_page(url)
    pdfs = filename_from_html(content)
    
    for pdf in pdfs:
        if check_year(pdf):
            print("Adding {} to download list".format(pdf))
            docs_to_download.append(pdf)
        else:
            pass

Adding SS18880101-01 to download list
Adding SS18880701-03 to download list
Adding SS18890101-01 to download list
Adding SS18890701-03 to download list
Adding SS18891001-04 to download list
Adding SS18900104-01 to download list
Adding SS18900301-01e to download list
Adding SS18900405-02 to download list
Adding SS18900415-02e1 to download list
Adding SS18900501-02e2 to download list
Adding SS18900705-03 to download list
Adding SS18901004-04 to download list
Adding SS18910103-01 to download list
Adding SS18910201-02 to download list
Adding SS18910601-03 to download list
Adding SS18911001-04 to download list
Adding SS18920101-01 to download list
Adding SS18920401-02 to download list
Adding SS18920701-03 to download list
Adding SS18921001-04 to download list
Adding SS18930101-01 to download list
Adding SS18930401-02 to download list
Adding SS18930701-03 to download list
Adding SS18931001-04 to download list
Adding SS18940101-01 to download list
Adding SS18940401-02 to download list
Adding 

Finally, I loop through all of the filenames, create the URL to the PDF, and use `requests` to download a copy of the document into my directory for processing.

In [11]:
for doc_name in docs_to_download:
    url = join(baseurl, "{}.pdf".format(doc_name))
    print(url)
    with open(join(download_directory, "{}.pdf".format(doc_name)), "wb") as file:
        # get request
        response = requests.get(url)
        # write to file
        file.write(response.content)

http://documents.adventistarchives.org/SSQ/SS18880101-01.pdf
http://documents.adventistarchives.org/SSQ/SS18880701-03.pdf
http://documents.adventistarchives.org/SSQ/SS18890101-01.pdf
http://documents.adventistarchives.org/SSQ/SS18890701-03.pdf
http://documents.adventistarchives.org/SSQ/SS18891001-04.pdf
http://documents.adventistarchives.org/SSQ/SS18900104-01.pdf
http://documents.adventistarchives.org/SSQ/SS18900301-01e.pdf
http://documents.adventistarchives.org/SSQ/SS18900405-02.pdf
http://documents.adventistarchives.org/SSQ/SS18900415-02e1.pdf
http://documents.adventistarchives.org/SSQ/SS18900501-02e2.pdf
http://documents.adventistarchives.org/SSQ/SS18900705-03.pdf
http://documents.adventistarchives.org/SSQ/SS18901004-04.pdf
http://documents.adventistarchives.org/SSQ/SS18910103-01.pdf
http://documents.adventistarchives.org/SSQ/SS18910201-02.pdf
http://documents.adventistarchives.org/SSQ/SS18910601-03.pdf
http://documents.adventistarchives.org/SSQ/SS18911001-04.pdf
http://documents.ad

[^1]: The library at Northeastern University provides a useful [overview](http://subjectguides.lib.neu.edu/textdatamining) of the different elements one must consider when seeking to mine content in subscription databases. Increasingly, library vendors are seeking to build "research environments" that provide "[workflow and analytical technology that breathes new life into the study of the social sciences and humanities](http://www.gale.com/primary-sources/platform)." It is my opinion that granting vendors control over both the documentary materials and the methods of analysis, however, would be a concerning development for the long-term health of scholarship in the humanities.

[^2]: The University of Michigan is [the archival home of the *Good Health* magazine](https://mirlyn.lib.umich.edu/Record/003935956#0) and [many have been digitized by Google](https://books.google.com/books?id=Hds1AQAAMAAJ&dq=%22Good+Health%22&source=gbs_navlinks_s). The Office of Archives and Research does include a few issues from 1937 and [one from 1942](http://documents.adventistarchives.org/Periodicals/HR/HR19420601-V77-06.pdf), which offers evidence of Kellogg's ongoing editorial work on the title.

[^3]: In recognition of the labor involved in creating the digital assets and to drive traffic back to their site, I am not redistributing any of the PDF files in my dissertation. Copies can be retrieved directly from the denomination's websites.

[^4]: In this syntax I was guided by Ian Milligan's wget lesson. Ian Milligan, "Automated Downloading with Wget," *Programming Historian*, (2012-06-27), http://programminghistorian.org/lessons/automated-downloading-with-wget